In [2]:
import pandas as pd

In [22]:
sql_command = 'https://api.opendota.com/api/explorer?sql=SELECT%0Amatches.match_id%2C%0Adire_team_id%2C%0Aradiant_team_id%2C%0Achat%0AFROM%20matches%0AWHERE%20TRUE%0AAND%20(matches.leagueid%20%3D%209870)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272018-07-27T23%3A20%3A18.192Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST'

In [12]:
import requests
def getHtml(url):
    try:
        r=requests.get(url,timeout=30)
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        return r.text
    except:
        print("err")
        return None

In [23]:
ti8chat = getHtml(sql_command)

In [84]:
import json
chat_json = json.loads(ti8chat)

In [136]:
chat_json['rows'][0]

{'chat': [{'key': '19002',
   'player_slot': 1,
   'slot': 1,
   'time': -47,
   'type': 'chatwheel'},
  {'key': 'hf',
   'player_slot': 131,
   'slot': 8,
   'time': -45,
   'type': 'chat',
   'unit': 'Gh'},
  {'key': '150',
   'player_slot': 132,
   'slot': 9,
   'time': -43,
   'type': 'chatwheel'},
  {'key': '123',
   'player_slot': 1,
   'slot': 1,
   'time': -42,
   'type': 'chatwheel'},
  {'key': 'HF',
   'player_slot': 1,
   'slot': 1,
   'time': -41,
   'type': 'chat',
   'unit': 'tokyo'},
  {'key': 'hfhf',
   'player_slot': 4,
   'slot': 4,
   'time': -39,
   'type': 'chat',
   'unit': 'universe'},
  {'key': '150',
   'player_slot': 132,
   'slot': 9,
   'time': -39,
   'type': 'chatwheel'},
  {'key': '150',
   'player_slot': 132,
   'slot': 9,
   'time': -35,
   'type': 'chatwheel'},
  {'key': '104',
   'player_slot': 1,
   'slot': 1,
   'time': -11,
   'type': 'chatwheel'},
  {'key': '19002',
   'player_slot': 1,
   'slot': 1,
   'time': 811,
   'type': 'chatwheel'},
  {'ke

In [85]:
chat_df = pd.DataFrame(chat_json['rows'])

In [86]:
chat_df.head()

,chat,dire_team_id,match_id,radiant_team_id
0,"[{'time': -47, 'type': 'chatwheel', 'key': '19...",2163,4061660411,350190
1,"[{'time': -84, 'type': 'chatwheel', 'key': '56...",5027210,4061662358,39
2,"[{'time': -131, 'type': 'chatwheel', 'key': '1...",15,4061671194,2586976
3,"[{'time': -77, 'type': 'chat', 'unit': 'jabz',...",5,4061694810,543897
4,"[{'time': -123, 'type': 'chatwheel', 'key': '1...",15,4061721528,2586976


In [164]:
def count_chatwheel(group):
#     for chatdict in group['chat'][0]:
#         print(chatdict)
     
    dire_range = range(0,5)
    num_dire_chatwheel, num_radiant_chatwheel = 0, 0
    for chatdict in group['chat']:
        for each_dict in chatdict:
            if each_dict['type'] == 'chatwheel':
                if each_dict['player_slot'] in dire_range:
                    num_dire_chatwheel += 1
                else:
                    num_radiant_chatwheel += 1

    group['num_dire_chatwheel'] = num_dire_chatwheel
    group['num_radiant_chatwheel'] = num_radiant_chatwheel#num_radiant_chatwheel
    return group

In [184]:
chat_test = chat_df.groupby('match_id').apply(count_chatwheel)
# chat_test

In [185]:
chat_dire =  chat_test[['dire_team_id','num_dire_chatwheel']]
chat_dire.columns = ['team_id','num_chatwheel']

chat_radiant =  chat_test[['radiant_team_id','num_radiant_chatwheel']]
chat_radiant.columns = ['team_id','num_chatwheel']
# chat_dire
chat_all = pd.concat([chat_radiant,chat_dire])

In [202]:
chat_all.groupby('team_id').agg([max,min,sum,'count','mean']).reset_index()

team_id num_chatwheel                           
                      max min   sum count       mean
0         5            78   2   314    18  17.444444
1        15           145   0  1019    30  33.966667
2        39           108   0   644    27  23.851852
3        67            45   1   186    16  11.625000
4      2163            68   1   333    24  13.875000
5    350190            53   1   267    17  15.705882
6    543897            53   0   373    19  19.631579
7    726228            55   3   219    20  10.950000
8   1375614            95   3   316    19  16.631579
9   1838315            89   0   303    26  11.653846
10  1883502            94   0   345    25  13.800000
11  2108395           168   0   455    17  26.764706
12  2586976            26   0   236    29   8.137931
13  5026801            82   1   502    24  20.916667
14  5027210            81   0   279    17  16.411765
15  5066616            32   1   237    19  12.473684
16  5228654           124   0   504    22  22.909091
17  5229127           101   1   330    21  15.714286

In [248]:
team_info = chat_all['team_id'].drop_duplicates()
team_info = pd.DataFrame(team_info)

In [259]:
def get_team_name(id):
    get_team_url = 'https://api.opendota.com/api/teams/{0}'.format(id)
    print(get_team_url)
    team_name = eval(getHtml(get_team_url))['name']
    return team_name
get_team_name(15)

https://api.opendota.com/api/teams/15


'PSG.LGD'

In [311]:
team_info['name']= team_info['team_id'].map(lambda x:get_team_name(x))

https://api.opendota.com/api/teams/350190
https://api.opendota.com/api/teams/39
https://api.opendota.com/api/teams/2586976
https://api.opendota.com/api/teams/543897
https://api.opendota.com/api/teams/5026801
https://api.opendota.com/api/teams/5228654
https://api.opendota.com/api/teams/1838315
https://api.opendota.com/api/teams/2108395
https://api.opendota.com/api/teams/5
https://api.opendota.com/api/teams/5229127
https://api.opendota.com/api/teams/15
https://api.opendota.com/api/teams/5027210
https://api.opendota.com/api/teams/2163
https://api.opendota.com/api/teams/67
https://api.opendota.com/api/teams/1883502
https://api.opendota.com/api/teams/726228
https://api.opendota.com/api/teams/5066616
https://api.opendota.com/api/teams/1375614


In [323]:
def get_now_players(team_id):
    players_url = 'https://api.opendota.com/api/teams/{0}/players'.format(team_id)
    now_player = {}
    import json 
    for each_player in json.loads(getHtml(players_url)):
#         print(each_player)
        if each_player['is_current_team_member'] == True:
            now_player[each_player['account_id']] = each_player['name']
    print(now_player)
    
get_now_players(15)

{106863163: 'Somnus丶M', 107081378: 'Victoria丶JY', 177416702: 'AME', 101695162: 'fy', 125581247: 'Ame', 94738847: 'Chalice', 94296097: 'xNova', 131237305: 'XZ', 118678359: 'Fei', 89399750: 'QQQ'}


In [312]:
team_info

,team_id,name
0,350190,Fnatic
1,39,Evil Geniuses
2,2586976,OG
3,543897,Mineski
8,5026801,OpTic Gaming
9,5228654,VGJ Storm
10,1838315,Team Secret
11,2108395,TNC Predator
16,5,Invictus Gaming
17,5229127,Winstrike


In [ ]:
team_info['team_id'].head(1).map()